In [66]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import os
import time
from sklearn import datasets
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score
from keras.datasets import cifar10
from keras.utils import to_categorical
import numpy as np

In [67]:
# load train and test dataset
def load_dataset():
    # load dataset
    data_x = np.load('data_x_original.npy')
    data_y = np.load('data_y_original.npy')
    #percentage of the dataset which will be on TRAIN - TEST - VALIDATION
    TOTAL_IMAGES = len(data_y)
    print("TOTAL_IMAGES = ", TOTAL_IMAGES)
    
    TRAIN_PERCENTAGE = 0.8
    TEST_PERCENTAGE = 0.1
    VALIDATION_PERCENTAGE = 0.1

    train_size = int(TRAIN_PERCENTAGE * TOTAL_IMAGES)
    test_size = int(TEST_PERCENTAGE * TOTAL_IMAGES)
    validation_size = int(VALIDATION_PERCENTAGE * TOTAL_IMAGES)

    train_x = data_x[:train_size] #taking train images from original data
    train_y = data_y[:train_size]
    data_x = data_x[train_size:] #removing train images from original data
    data_y = data_y[train_size:]

    test_x = data_x[:test_size] #taking test images from original data
    test_y = data_y[:test_size]
    data_x = data_x[test_size:] #removing test images from original data
    data_y = data_y[test_size:]

    validation_x = data_x #all the remainder images we can put on validation
    validation_y = data_y

    del data_x #free some unused memory
    del data_y

    print("train_size = ", len(train_y))
    print("test_size = ", len(test_y))
    print("validation_size = ", len(validation_y))
 # one hot encode target values
    train_y = to_categorical(train_y)
    test_y = to_categorical(test_y)
    validation_y = to_categorical(validation_y)
    return train_x, train_y, test_x, test_y, validation_x, validation_y

In [68]:
# scale pixels
def prep_pixels(train, test, validation):
    # convert from integers to floats
    train_norm = train.astype('float32')
    test_norm = test.astype('float32')
    validation_norm = validation.astype('float32')
    # normalize to range 0-1
    train_norm = train_norm / 255.0
    test_norm = test_norm / 255.0
    validation_norm = validation_norm / 255.0
    # return normalized images
    return train_norm, test_norm, validation_norm

In [69]:
trainX, trainY, testX, testY, validationX, validationY = load_dataset()
trainX, testX, validationX = prep_pixels(trainX, testX, validationX)

TOTAL_IMAGES =  592
train_size =  473
test_size =  59
validation_size =  60


In [70]:
trainX[0]

array([[[0.50980395, 0.47058824, 0.6039216 ],
        [0.5921569 , 0.5882353 , 0.6509804 ],
        [0.7294118 , 0.70980394, 0.7490196 ],
        ...,
        [0.6862745 , 0.6313726 , 0.7254902 ],
        [0.56078434, 0.49803922, 0.6509804 ],
        [0.48235294, 0.42352942, 0.59607846]],

       [[0.65882355, 0.6431373 , 0.70980394],
        [0.73333335, 0.7490196 , 0.7529412 ],
        [0.8235294 , 0.827451  , 0.81960785],
        ...,
        [0.57254905, 0.5294118 , 0.6392157 ],
        [0.48235294, 0.42745098, 0.57254905],
        [0.45490196, 0.38039216, 0.5647059 ]],

       [[0.8352941 , 0.8       , 0.8235294 ],
        [0.8627451 , 0.85882354, 0.84705883],
        [0.87058824, 0.8627451 , 0.8509804 ],
        ...,
        [0.5058824 , 0.45490196, 0.60784316],
        [0.49411765, 0.41960785, 0.6       ],
        [0.48235294, 0.40392157, 0.6       ]],

       ...,

       [[0.7058824 , 0.67058825, 0.73333335],
        [0.79607844, 0.7647059 , 0.78039217],
        [0.83137256, 0

In [71]:
clf1 = tf.keras.models.load_model("displasys_mobilenet-baseline-holdout-100epochs_dataAugmenting_l2_dropout.npy")

In [72]:
print(type(clf1))

<class 'keras.engine.functional.Functional'>


In [73]:
clf2 = tf.keras.models.load_model("displasys_resnet50v2-baseline-holdout-100epochs_dataAugmenting_l2_dropout.npy")

In [74]:
#clf3 = tf.keras.models.load_model("resnet50-baseline-holdout-100epochs_normalized.npy")

In [75]:
class emsembleModel:
    def __init__(self, classifiers, numClasses):
        self.classifiers = classifiers
        self.numClasses = numClasses
        
    def predict_by_voting(self, dataX):
        preds = []
        for clf in self.classifiers:
            preds.append(clf.predict(dataX))
        
        resultPred = []
        M = len(self.classifiers)
        for i in range(len(dataX)):
            counting = [0 for _ in range(self.numClasses)]
            current_best_pred = preds[0][i]
            for j in range(M):
                pred = preds[j][i]
                predictedClass = np.argmax(pred)
                counting[predictedClass]+=1
                if counting[predictedClass] == max(counting):
                    current_best_pred = pred
            resultPred.append(current_best_pred)
        
        return resultPred
    
    def evaluate_by_voting(self, dataX, dataY):
        pred = self.predict_by_voting(dataX)
        ok = 0
        for i in range(len(pred)):
            expectedClass = np.argmax(dataY[i])
            predictedClass = np.argmax(pred[i])
            if predictedClass == expectedClass:
                ok += 1
        return ok/len(dataY)

    def predict_by_softmax(self, dataX):
        preds = []
        for clf in self.classifiers:
            preds.append(clf.predict(dataX))
        print(preds[0].shape, preds[1].shape)
        resultsPred = []
        M = len(self.classifiers)
        for i in range(len(dataX)):
            probs = []
            for j in range(M):
                pred = preds[j][i]
                probs.append(pred)
            compressed_probs = np.array(probs).sum(axis=0)
            compressed_probs = compressed_probs/M
            resultsPred.append(compressed_probs)
        
        return resultsPred
    
    def evaluate_by_softmax(self, dataX, dataY):
        pred = self.predict_by_softmax(dataX)
        ok = 0
        for i in range(len(pred)):
            expectedClass =  np.argmax(dataY[i])
            predictedClass = np.argmax(pred[i])
            #print("predictedClass = ", predictedClass)
            #print("expectedClass = ", expectedClass)
            if predictedClass == expectedClass:
                ok += 1
        return ok/len(dataY)
    

In [76]:
# classifiers = [clf1, clf2, clf3]
# classifiers = [clf1, clf2]
classifiers = [clf2, clf3]

In [77]:
emsemble = emsembleModel(classifiers, 4)

In [78]:
acc = emsemble.evaluate_by_softmax(testX, testY)

2/2 [==============================] - 0s 87ms/step
(59, 4, 4, 2048) (59, 4, 4, 2048)


In [79]:
print('Accuracy Emsemble using Softmax: %.3f' % (acc * 100.0))

Accuracy Emsemble using Softmax: 0.000


In [80]:
acc = emsemble.evaluate_by_voting(testX, testY)

2/2 [==============================] - 0s 85ms/step


IndexError: list index out of range

In [ ]:
print('Accuracy Emsemble using Voting: %.3f' % (acc * 100.0))